# Ferramentas do Projeto

<p>Para analisar a base de dados, podemos utilizar a biblioteca pandas, que se adequa bem ao tipo de análise que desejamos realizar.</p> 

In [ ]:
import pandas as pd

# Extracão e Tratamento dos dados

<p>Uma vez importada a biblioteca, utilizamos o comando read_csv(), utilizando o separador apropriado do arquivo csv disponibilizado, para ler os dados e inserí-los num dataframe.</p>

In [ ]:
database = pd.read_csv('./DB_Teste.csv', sep=';')
database.head()

<p>Tendo a base de dados em mãos, é preciso realizar um tratamento dos dados recebidos, para garantir que não há problemas com nossos dados, e, caso haja, corrigí-los antes que qualquer análise seja feita.</p>

<p>Primeiramente vamos retirar informacões a respeito do dataframe com a funcão info()</p>

In [ ]:
print(database.info())

<p>Temos uma coluna, a 'Unnamed:10', que não possui nenhum valor não nulo. Podemos retirá-la sem prejuízo à análise.</p>

In [ ]:
del database['Unnamed: 10']
database

<p>Analisando os tipos de dados de cada coluna, vemos que a coluna de valor não tem um tipo numérico de dado. Portanto, precisamos converter esta coluna em uma coluna de floats</p>

In [ ]:
database['Valor'] = database['Valor'].replace('[R$.]', '', regex=True).str.replace(',','.').astype(float)
print(database.info())
database

<p>Por último, devemos checar se não há nenhuma linha duplicada, por precaucão. Podemos inferir, pelo tipo de base de dados que temos, que a única coluna que diferencia as entradas umas das outras é a coluna ID, pois as outras podem possuir mais de uma ocorrência em mais de uma linha sem problemas.</p>

In [ ]:
# devolve todas as linhas duplicadas com base na ID
linhas_duplicadas = database.duplicated(subset=['ID'], keep=False) 
# any(linhas_duplicadas) vai ser true se houver pelo menos uma linha duplicada. 
if any(linhas_duplicadas):
    print(database[linhas_duplicadas])

<p>Podemos notar que uma ID foi utilizada em duas entradas da base de dados. Podemos colocá-la numa variavel separada para usar posteriormente em nossas análises, por questões de sanity check.</p>

In [ ]:
ID_duplicada = database.loc[linhas_duplicadas, 'ID'].values[0]
print(ID_duplicada)

<p>Agora que temos os dados do jeito que queríamos, podemos partir para a análise</p>

# Análise


### 1) Construa uma tabela auxiliar que sumarize o valor vendido por cada vendedor, ordenando do maior para o menor

In [ ]:
# Resposta:

# primeiramente criamos a tabela agrupando por vendedor e somando todos os valores de suas vendas
resumo_vendas = database.groupby('Vendedor')['Valor'].sum().reset_index()
# retirando a notacao cientifica do output, para faciliar a leitura dos numeros
pd.set_option('display.float_format', lambda x: '%.2f' % x) 

# em seguida ordenamos a tabela em ordem descrescente
resumo_vendas = resumo_vendas.sort_values(by='Valor', ascending=False)
resumo_vendas = resumo_vendas.rename(columns={'Valor': 'Valor Vendido'})
print(resumo_vendas)

### 2) Imprima e identifica qual foi o cliente responsável pela venda com maior valor e com menor valor

In [ ]:
# podemos encontrar o cliente responsável pela maior venda aplicando loc no valor, 
# filtrando pelo id atribuido ao maior valor e pegando o valor contido na coluna cliente. 
cliente_maior_venda = database.loc[database['Valor'].idxmax(), 'Cliente']
print(f"O cliente responsável pela maior venda foi o {cliente_maior_venda}")

# o mesmo raciocínio é válido para a menor venda
cliente_menor_venda = database.loc[database['Valor'].idxmin(), 'Cliente']
print(f"Já o cliente responsável pela menor venda foi o {cliente_menor_venda}")

### 3) Imprima valor médio por Tipo de venda (Serviços, Licenciamento, Produtos)

In [ ]:
# Agrupando por tipo e aplicando a media obtemos o resultado
valor_medio_tipo = database.groupby('Tipo')['Valor'].mean().reset_index()
valor_medio_tipo['Valor'] = valor_medio_tipo['Valor'].round(2)
valor_medio_tipo = valor_medio_tipo.rename(columns={'Valor': 'Valor Medio por Tipo'})
valor_medio_tipo

### 4) Imprima o número de vendas realizada por cliente

In [ ]:
# aplicando a funcao value_counts() na coluna Cliente obtemos o número de entradas por cliente
numero_vendas_por_cliente = database['Cliente'].value_counts().reset_index()
numero_vendas_por_cliente.columns = ['Cliente', 'Numero de Vendas']
numero_vendas_por_cliente

# GRÁFICO DA PARTE 2 DO DESAFIO

In [ ]:
import matplotlib.pyplot as plt

<p></p>

In [ ]:
database['Data da Venda'] = pd.to_datetime(database['Data da Venda'], format='%d/%m/%Y')
database['Trimestre'] = database['Data da Venda'].dt.to_period('Q') # 'Q' define um trimestre (quarto de ano)

In [ ]:
database.head()

In [ ]:
soma_por_trimestre = database.groupby('Trimestre')['Valor'].sum().reset_index()
# convertendo para string para evitar erros com o plot
soma_por_trimestre['Trimestre'] = soma_por_trimestre['Trimestre'].astype(str) 
print(soma_por_trimestre)

In [ ]:
plt.plot(soma_por_trimestre['Trimestre'], soma_por_trimestre['Valor'], marker='o')
plt.title('Valor Total de Vendas por Trimestre')
plt.xlabel('Trimestre')
plt.ylabel('Valor Total de Vendas por Trimestre')
 # retirando a notacao cientifica do eixo y para melhor legibilidade
plt.ticklabel_format(style='plain', axis='y')
plt.xticks(rotation=45) # rotacionanco os valores no eixo x para melhor legibilidade
plt.show()